In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
final_match = pd.read_csv("/content/drive/MyDrive/ML_project/matches_with_points.csv")
# final_match2 = pd.read_csv("/content/drive/MyDrive/ML_project/matches_with_average_points.csv")

player_points = pd.read_csv("/content/drive/MyDrive/ML_project/player_points.csv")
team_player = pd.read_csv("/content/drive/MyDrive/ML_project/team_player_df.csv")

In [ ]:
final_match.head()

In [ ]:
player_points.head(20)

In [ ]:
team_player.head()

In [ ]:
final_match = final_match.loc[:, ~final_match.columns.str.contains('^Unnamed')]
final_match.head()

In [ ]:
ActualNames = ['Sunrisers Hyderabad', 'Mumbai Indians', 'Gujarat Lions', 'Rising Pune Supergiant', 'Royal Challengers Bangalore', 'Kolkata Knight Riders', 'Delhi Daredevils', 'Kings XI Punjab', 'Chennai Super Kings', 'Rajasthan Royals', 'Deccan Chargers',
    'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants', 'Delhi Capitals']

ChangedNames = ['SRH','MI','GL','RPS','RCB','KKR','DC','KXIP','CSK','RR','SRH','KTK','PW','RPS','DC']

final_match.replace(ActualNames, ChangedNames, inplace = True)

In [ ]:
nameChangedct = {}
for name in range(len(ActualNames)):
    nameChangedct[ActualNames[name]] = ChangedNames[name]

team_player.rename(nameChangedct, axis='columns', inplace=True)
team_player = team_player.loc[:, ~team_player.columns.str.contains('^Unnamed')]
team_player.head(15)

In [ ]:
matches_pred = final_match.drop(columns=['id', 'player_of_match', 'venue', 'umpire1', 'umpire2', 'result_margin'])
matches_pred.head()


In [ ]:
player_points.columns.values[0] = "year"
player_points.fillna(0, inplace=True)
player_points.head()

In [ ]:
# Regression
from sklearn.linear_model import Lasso

reg_data = player_points.copy(deep=True)
player_points_prediction = {}

test_all = []
pred_all = []

for colNum in range(1, len(player_points.columns)):
    df = reg_data.iloc[: , [0, colNum]].copy() 

    train = df[df["year"] != 2020]
    test = df[df["year"] == 2020]
    
    x_train = train["year"].values.reshape(-1, 1)
    
    y_train = train[player_points.columns[colNum]].values.reshape(-1, 1)
    x_test = test["year"].values.reshape(1, -1)
    y_test = test[player_points.columns[colNum]].values.reshape(1, -1)

    test_all.append(y_test[0][0])

    model = Lasso(alpha = 0.1)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    pred_all.append(y_pred[0])

    player_points_prediction[player_points.columns[colNum]] = y_pred[0]
    



In [ ]:
# Calculating mse, mae and rmse
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

print(mean_absolute_error(test_all, pred_all))
print(mean_squared_error(test_all, pred_all, squared=True))
print(mean_absolute_percentage_error(test_all, pred_all))
print(mean_squared_error(test_all, pred_all, squared=False))

In [ ]:
teams = matches_pred['team1'].unique()
teams

In [ ]:
team_2020_points = {}

for team_name in teams:
    tot_pts = 0
    team_members = team_player.iloc[-1][team_name]
    if type(team_members) is str:
        team_members = team_members.strip("{}")
        for player in team_members.split(", "):
            try:
                tot_pts += player_points_prediction[player.strip("'")]
            except KeyError:
                tot_pts += 0
    
        # team_2020_points[team_name] = tot_pts/len(team_members.split(", "))

    # else:
        # team_2020_points[team_name] = 0
    team_2020_points[team_name] = tot_pts
    




In [ ]:
for i in range(matches_pred[matches_pred["yr"] != 2020].shape[0], matches_pred.shape[0]):
  matches_pred.at[i, 'team1_points'] =  float(team_2020_points[matches_pred.iloc[i]['team1']])
  matches_pred.at[i, 'team2_points'] =  float(team_2020_points[matches_pred.iloc[i]['team2']])

In [ ]:
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder

# encodedct = defaultdict(LabelEncoder)
# toEncode = ["city", "toss_winner", "toss_decision", "result"]
# matches_pred_encoded = matches_pred.copy(deep = True)

# for colName in toEncode:
#     matches_pred_encoded[colName] = encodedct[colName].fit_transform(matches_pred[colName])

# matches_pred_encoded["winner"] = encodedct["toss_winner"].transform(matches_pred_encoded["winner"])

encoder = LabelEncoder()
matches_pred["team1"] = encoder.fit_transform(matches_pred["team1"])
matches_pred["team2"] = encoder.fit_transform(matches_pred["team2"])
matches_pred["winner"] = encoder.fit_transform(matches_pred["winner"].astype(str))
matches_pred["toss_winner"] = encoder.fit_transform(matches_pred["toss_winner"])

matches_pred['city'] = encoder.fit_transform(matches_pred['city'])
matches_pred["toss_decision"] = encoder.fit_transform(matches_pred["toss_decision"])

matches_pred["result"] = encoder.fit_transform(matches_pred["result"])


matches_pred.head()

,city,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,yr,team1_points,team2_points
0,2,0,8,3,8,1,3,0,2008,1595.5,1535.5
1,7,0,5,0,0,0,0,0,2008,2006.5,2032.0
2,10,0,1,10,10,0,1,1,2008,1796.5,2214.5
3,23,0,6,8,6,0,8,1,2008,1849.5,1595.5
4,22,0,3,11,11,0,3,1,2008,1535.5,1738.0


In [ ]:
# toEncodeAlso = ["team1", "team2"]
# for colName in toEncodeAlso:
#     matches_pred_encoded[colName] = encodedct["toss_winner"].transform(matches_pred[colName])

# matches_pred_encoded["yr"] = encodedct["yr"].fit_transform(matches_pred["yr"])


In [ ]:
matches_pred.loc[matches_pred["winner"] == matches_pred["team1"],"team1_win"] = 1
matches_pred.loc[matches_pred["winner"] != matches_pred["team1"],"team1_win"] = 0

matches_pred.loc[matches_pred["toss_winner"] == matches_pred["team1"],"team1_toss_win"] = 1
matches_pred.loc[matches_pred["toss_winner"] != matches_pred["team1"],"team1_toss_win"] = 0


In [ ]:
train = matches_pred[matches_pred['yr'] != 2020]
test = matches_pred[matches_pred['yr'] == 2020]

In [ ]:
train_y = train.team1_win.values
del train["team1_win"]
del train["winner"]
del train['yr']
del train["toss_winner"]
train_X = train.values.astype(np.float)

In [ ]:
test_y = test.team1_win.values
del test["team1_win"]
del test["winner"]
del test["toss_winner"]
del test['yr']
test_X = test.values.astype(np.float)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score, f1_score, recall_score, accuracy_score

In [ ]:
logreg = LogisticRegression(max_iter = 1000)
logreg.fit(train_X, train_y)
pred_y = logreg.predict(test_X)

print('Accuracy of Logistic Regression Classifier on test set: {:.4f}'.format(logreg.score(test_X, test_y)))

print(precision_score(pred_y, test_y))
print(f1_score(pred_y, test_y))
print(recall_score(pred_y, test_y))

cm = confusion_matrix(test_y, pred_y, labels = logreg.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = logreg.classes_)
disp.plot()
plt.show()


In [ ]:
from sklearn import svm

model = svm.SVC()
model.fit(train_X, train_y)
pred_y = model.predict(test_X)

print('Accuracy of SVM Classifier on test set: {:.4f}'.format(accuracy_score(test_y, pred_y)))

print(precision_score(pred_y, test_y))
print(f1_score(pred_y, test_y))
print(recall_score(pred_y, test_y))

cm = confusion_matrix(test_y, pred_y, labels = logreg.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = logreg.classes_)
disp.plot()
plt.show()



In [ ]:
from sklearn import tree

model = tree.DecisionTreeClassifier()
model.fit(train_X, train_y)
pred_y = model.predict(test_X)

print('Accuracy of Decision Tree Classifier on test set: {:.4f}'.format(accuracy_score(test_y, pred_y)))

print(precision_score(pred_y, test_y))
print(f1_score(pred_y, test_y))
print(recall_score(pred_y, test_y))

cm = confusion_matrix(test_y, pred_y, labels = logreg.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = logreg.classes_)
disp.plot()
plt.show()



In [ ]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(n_estimators=100)
model.fit(train_X, train_y)
pred_y = model.predict(test_X)

print('Accuracy of AdaBoost Classifier on test set: {:.4f}'.format(accuracy_score(test_y, pred_y)))

print(precision_score(pred_y, test_y))
print(f1_score(pred_y, test_y))
print(recall_score(pred_y, test_y))

cm = confusion_matrix(test_y, pred_y, labels = logreg.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = logreg.classes_)
disp.plot()
plt.show()



In [ ]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(solver='lbfgs', max_iter = 50000)
model.fit(train_X, train_y)
pred_y = model.predict(test_X)

print('Accuracy of MLP Classifier on test set: {:.4f}'.format(accuracy_score(test_y, pred_y)))

print(precision_score(pred_y, test_y))
print(f1_score(pred_y, test_y))
print(recall_score(pred_y, test_y))

cm = confusion_matrix(test_y, pred_y, labels = logreg.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = logreg.classes_)
disp.plot()
plt.show()

